In [0]:
customers_df = spark.read.csv("/Volumes/workspace/default/food_delivery/customers.csv", header=True, inferSchema=True)
restaurants_df = spark.read.csv("/Volumes/workspace/default/food_delivery/orders.csv", header=True, inferSchema=True)
delivery_partners_df = spark.read.csv("/Volumes/workspace/default/food_delivery/delivery_partners.csv", header=True, inferSchema=True)
orders_df = spark.read.csv("/Volumes/workspace/default/food_delivery/orders.csv", header=True, inferSchema=True)

display(customers_df.limit(5))
display(restaurants_df.limit(5))
display(delivery_partners_df.limit(5))
display(orders_df.limit(5))


customer_id,customer_name,locality
C001,Arjun Mehta,Delhi
C002,Ananya Sharma,Mumbai
C003,Rahul Verma,Bangalore
C004,Neha Gupta,Hyderabad
C005,Amit Singh,Chennai


order_id,customer_id,restaurant_id,delivery_partner_id,order_value,delivery_time,order_status,date_timestamp
O001,C001,R001,D001,450.75,32,Delivered,2025-08-01T12:15:00.000Z
O002,C005,R004,D004,799.5,40,Delivered,2025-08-01T13:05:00.000Z
O003,C010,R007,D009,299.0,25,Cancelled,2025-08-01T14:10:00.000Z
O004,C012,R002,D002,560.2,30,Delivered,2025-08-01T18:45:00.000Z
O005,C015,R003,D003,999.99,55,Delivered,2025-08-02T12:00:00.000Z


delivery_partner_id,delivery_partner_name,locality
D001,Ajay Kumar,Delhi
D002,Ramesh Singh,Mumbai
D003,Vinod Sharma,Bangalore
D004,Santosh Gupta,Hyderabad
D005,Arvind Reddy,Chennai


order_id,customer_id,restaurant_id,delivery_partner_id,order_value,delivery_time,order_status,date_timestamp
O001,C001,R001,D001,450.75,32,Delivered,2025-08-01T12:15:00.000Z
O002,C005,R004,D004,799.5,40,Delivered,2025-08-01T13:05:00.000Z
O003,C010,R007,D009,299.0,25,Cancelled,2025-08-01T14:10:00.000Z
O004,C012,R002,D002,560.2,30,Delivered,2025-08-01T18:45:00.000Z
O005,C015,R003,D003,999.99,55,Delivered,2025-08-02T12:00:00.000Z


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

# Initialize Spark Session (if not already done)
spark = SparkSession.builder.appName("FoodDeliveryAnalysis").getOrCreate()

# Load the orders.csv file from the Databricks file system (DBFS)
df_orders = spark.read.csv("/Volumes/workspace/default/food_delivery/orders.csv", header=True, inferSchema=True)

# Compute the total order value for each customer
total_order_value_per_customer = df_orders.groupBy("customer_id") \
    .agg(sum("order_value").alias("total_order_value")) \
    .sort("total_order_value", ascending=False)

# Display the result
total_order_value_per_customer.display()

customer_id,total_order_value
C035,1299.0
C015,999.99
C013,999.5
C018,990.0
C038,955.4
C041,935.9
C045,890.9
C024,875.4
C019,875.0
C017,870.5


Databricks visualization. Run in Databricks to view.

question 2

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

# Initialize Spark Session (if not already done)
spark = SparkSession.builder.appName("FoodDeliveryAnalysis").getOrCreate()

# Load the orders.csv file from the Databricks file system (DBFS)
df_orders = spark.read.csv("/Volumes/workspace/default/food_delivery/orders.csv", header=True, inferSchema=True)

# Calculate the average delivery time for each restaurant
avg_delivery_time_per_restaurant = df_orders.groupBy("restaurant_id") \
    .agg(avg("delivery_time").alias("average_delivery_time")) \
    .sort("average_delivery_time", ascending=False)

# Display the result
avg_delivery_time_per_restaurant.display()

restaurant_id,average_delivery_time
R006,65.0
R014,60.0
R034,58.0
R003,55.0
R027,55.0
R044,54.0
R022,52.0
R048,50.0
R005,50.0
R030,49.0


Databricks visualization. Run in Databricks to view.

In [0]:
%python
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("FrequentCancellations").getOrCreate()

# Load the orders.csv file from the Databricks file system (DBFS)
df_orders = spark.read.csv("/Volumes/workspace/default/food_delivery/orders.csv", header=True, inferSchema=True)

# Register the DataFrame as a temporary SQL view
df_orders.createOrReplaceTempView("orders")

# Use PySpark SQL to find customers with the most cancellations
sql_query_cancellations = """
SELECT
    customer_id,
    COUNT(*) AS total_cancellations
FROM
    orders
WHERE
    order_status = 'Cancelled'
GROUP BY
    customer_id
ORDER BY
    total_cancellations DESC
"""

# Execute the SQL query and show the result
frequent_cancellations = spark.sql(sql_query_cancellations)
display(frequent_cancellations)

customer_id,total_cancellations
C021,1
C030,1
C016,1
C042,1
C010,1
C020,1
C050,1
C031,1
C043,1


Databricks visualization. Run in Databricks to view.

sql question 2

In [0]:
%python
# Load the orders.csv file from the Databricks file system (DBFS)
df_orders = spark.read.csv(
    "/Volumes/workspace/default/food_delivery/orders.csv",
    header=True,
    inferSchema=True
)

# Register the DataFrame as a temporary SQL view
df_orders.createOrReplaceTempView("orders")

# Use PySpark SQL to find the peak order hours
sql_query_peak_hours = """
SELECT
    HOUR(date_timestamp) AS order_hour,
    COUNT(*) AS total_orders
FROM
    orders
GROUP BY
    order_hour
ORDER BY
    total_orders DESC
"""

# Execute the SQL query and show the result
peak_order_hours = spark.sql(sql_query_peak_hours)
display(peak_order_hours)

order_hour,total_orders
12,8
19,8
20,7
21,7
13,6
11,5
14,4
18,2
22,1
16,1


Databricks visualization. Run in Databricks to view.